In [ ]:
#三色比例图，直接更改底部main函数中的参数

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import ast
from scipy import stats
import json

def calculate_phase_metrics(log_ranks, log_vals, zipf_line, plateau_log_rank, elbow_log_rank):
    metrics = {}
    
    plateau_idx = np.where(log_ranks >= plateau_log_rank)[0][0] if any(log_ranks >= plateau_log_rank) else 0
    elbow_idx = np.where(log_ranks >= elbow_log_rank)[0][0] if any(log_ranks >= elbow_log_rank) else len(log_ranks)
    
    plateau_mask = np.arange(len(log_ranks)) < plateau_idx
    linear_mask = (np.arange(len(log_ranks)) >= plateau_idx) & (np.arange(len(log_ranks)) < elbow_idx)
    fast_decay_mask = np.arange(len(log_ranks)) >= elbow_idx
    
    metrics["plateau_size"] = np.sum(plateau_mask)
    metrics["linear_size"] = np.sum(linear_mask)
    metrics["fast_decay_size"] = np.sum(fast_decay_mask)
    
    total = len(log_ranks)
    metrics["plateau_percent"] = 100 * metrics["plateau_size"] / total
    metrics["linear_percent"] = 100 * metrics["linear_size"] / total
    metrics["fast_decay_percent"] = 100 * metrics["fast_decay_size"] / total
    
    if np.any(plateau_mask):
        metrics["plateau_area"] = np.sum(log_vals[plateau_mask] - zipf_line[plateau_mask])
    else:
        metrics["plateau_area"] = 0
        
    if np.any(fast_decay_mask):
        metrics["fast_decay_area"] = np.sum(zipf_line[fast_decay_mask] - log_vals[fast_decay_mask])
    else:
        metrics["fast_decay_area"] = 0
    
    if np.sum(plateau_mask) >= 2:
        plateau_slope, _, _, _, _ = stats.linregress(
            log_ranks[plateau_mask], log_vals[plateau_mask]
        )
        metrics["plateau_slope"] = plateau_slope
    else:
        metrics["plateau_slope"] = None
        
    if np.sum(linear_mask) >= 2:
        linear_slope, _, _, _, _ = stats.linregress(
            log_ranks[linear_mask], log_vals[linear_mask]
        )
        metrics["linear_slope"] = linear_slope
    else:
        metrics["linear_slope"] = None
        
    if np.sum(fast_decay_mask) >= 2:
        fast_decay_slope, _, _, _, _ = stats.linregress(
            log_ranks[fast_decay_mask], log_vals[fast_decay_mask]
        )
        metrics["fast_decay_slope"] = fast_decay_slope
    else:
        metrics["fast_decay_slope"] = None
    
    return metrics

def plot_zipf_with_fixed_transitions(
    log_ranks, log_vals, zipf_line, coeffs, 
    plateau_log_rank=3.5, elbow_log_rank=5.5,
    title="", save_path=None, ylim=None
):
    plt.rcParams.update({
        'font.size': 20,
        'axes.titlesize': 28,
        'axes.labelsize': 25,
        'xtick.labelsize': 20,
        'ytick.labelsize': 20,
        'legend.fontsize': 20
    })
    sns.set_palette("tab10")
    
    plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    
    plt.plot(log_ranks, log_vals, '.', alpha=0.6)
    
    plateau_idx = np.where(log_ranks >= plateau_log_rank)[0][0] if any(log_ranks >= plateau_log_rank) else 0
    elbow_idx = np.where(log_ranks >= elbow_log_rank)[0][0] if any(log_ranks >= elbow_log_rank) else len(log_ranks)
    
    plateau_info = {
        "plateau_detected": True,
        "index": plateau_idx,
        "log_rank": log_ranks[plateau_idx] if plateau_idx < len(log_ranks) else log_ranks[-1],
        "log_value": log_vals[plateau_idx] if plateau_idx < len(log_vals) else log_vals[-1]
    }
    
    elbow_info = {
        "elbow_detected": True,
        "index": elbow_idx,
        "log_rank": log_ranks[elbow_idx] if elbow_idx < len(log_ranks) else log_ranks[-1],
        "log_value": log_vals[elbow_idx] if elbow_idx < len(log_vals) else log_vals[-1]
    }
    
    metrics = calculate_phase_metrics(log_ranks, log_vals, zipf_line, plateau_log_rank, elbow_log_rank)
    
    print(f"Plateau phase neurons: {metrics['plateau_size']} ({metrics['plateau_percent']:.2f}%)")
    print(f"Linear phase neurons: {metrics['linear_size']} ({metrics['linear_percent']:.2f}%)")
    print(f"Fast decay phase neurons: {metrics['fast_decay_size']} ({metrics['fast_decay_percent']:.2f}%)")
    
    plt.axvspan(min(log_ranks), plateau_info["log_rank"], alpha=0.1, color='blue')
    plt.axvspan(plateau_info["log_rank"], elbow_info["log_rank"], alpha=0.1, color='green')
    plt.axvspan(elbow_info["log_rank"], max(log_ranks), alpha=0.1, color='red')
    
    bottom_y = min(log_vals) - (max(log_vals) - min(log_vals)) * 0.05
    
    plt.text(min(log_ranks) + (plateau_info["log_rank"] - min(log_ranks))/2, 
             bottom_y, 
             f"{metrics['plateau_percent']:.1f}%", 
             ha='center', va='top', color='blue', fontweight='bold')
    
    plt.text(plateau_info["log_rank"] + (elbow_info["log_rank"] - plateau_info["log_rank"])/2, 
             bottom_y, 
             f"{metrics['linear_percent']:.1f}%", 
             ha='center', va='top', color='green', fontweight='bold')
    
    plt.text(elbow_info["log_rank"] + (max(log_ranks) - elbow_info["log_rank"])/2, 
             bottom_y, 
             f"{metrics['fast_decay_percent']:.1f}%", 
             ha='center', va='top', color='red', fontweight='bold')
    
    plt.scatter(plateau_info["log_rank"], plateau_info["log_value"], 
                c='blue', marker='D', s=100)
    plt.scatter(elbow_info["log_rank"], elbow_info["log_value"], 
                c='green', marker='X', s=100)
    
    plt.hlines(plateau_info["log_value"], 
               xmin=min(log_ranks), xmax=plateau_info["log_rank"],
               colors='blue', linestyles='--', linewidth=3.5)
    plt.hlines(elbow_info["log_value"], 
               xmin=min(log_ranks), xmax=elbow_info["log_rank"],
               colors='green', linestyles='--', linewidth=3.5)
    
    plt.xlabel('Log Rank')
    plt.ylabel('Log Value')
    
    if ylim is not None:
        plt.ylim(ylim)
    
    if save_path:
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

def simplified_analysis(
    root_dir, 
    output_dir,
    target_step="114000",
    plateau_log_rank=3.5,
    elbow_log_rank=5.5,
    ylim=(-12, -3)
):
    root_path = Path(root_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    csv_files = list(root_path.rglob("500_all.csv"))
    print(f"Found {len(csv_files)} CSV files")
    
    results = {}
    
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df["abs_delta_loss_post_ablation"] = df["abs_delta_loss_post_ablation"].apply(ast.literal_eval)
        df["step"] = df["step"].astype(str)
        
        rel_id = str(csv_file.relative_to(root_path).parent).replace("/", "_")
        
        step_data = df[df["step"] == target_step]
        
        if len(step_data) == 0:
            print(f"Warning: Step {target_step} not found in {rel_id}")
            continue
        
        for _, row in step_data.iterrows():
            vals = row["abs_delta_loss_post_ablation"]
            
            arr = np.array(vals)
            arr = arr[arr > 0]
            sorted_vals = np.sort(arr)[::-1]
            ranks = np.arange(1, len(sorted_vals)+1)
            log_ranks = np.log(ranks)
            log_vals = np.log(sorted_vals)
            
            coeffs = np.polyfit(log_ranks, log_vals, deg=1)
            zipf_line = np.poly1d(coeffs)(log_ranks)
            
            title = f"{rel_id}"
            save_path = output_path / f"{rel_id}_step{target_step}_zipf_transitions.png"
            
            plot_zipf_with_fixed_transitions(
                log_ranks, 
                log_vals, 
                zipf_line, 
                coeffs, 
                plateau_log_rank=plateau_log_rank,
                elbow_log_rank=elbow_log_rank,
                title=title,
                save_path=save_path,
                ylim=ylim
            )
            
            metrics = calculate_phase_metrics(
                log_ranks, 
                log_vals, 
                zipf_line, 
                plateau_log_rank, 
                elbow_log_rank
            )
            
            results[rel_id] = {
                "zipf_slope": coeffs[0],
                "plateau_percent": metrics["plateau_percent"],
                "linear_percent": metrics["linear_percent"],
                "fast_decay_percent": metrics["fast_decay_percent"]
            }
    
    summary_path = output_path / "phase_summary.json"
    with open(summary_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    return results

if __name__ == "__main__":
    results = simplified_analysis(
        root_dir="/Users/curiostudio/Desktop/apd/gpt2",
        output_dir="/Users/curiostudio/Desktop/apd/",
        target_step="last",
        plateau_log_rank=3.5,
        elbow_log_rank=5.5,
        ylim=(-12, -3)
    )

In [ ]:
#线性拟合+截距检测
#直接调整Config里的信息就行

# Configuration Parameters - Easy to modify in one place
class LogRankConfig:
    # Log rank range for linear regression
    LOG_RANK_MIN = 4.8  # Minimum log rank value for target range
    LOG_RANK_MAX = 5.2  # Maximum log rank value for target range
    
    # Output settings
    SAVE_INDIVIDUAL_RESULTS = True   # Whether to save individual file results
    SAVE_CONSOLIDATED_RESULTS = True # Whether to save consolidated results
    
    # Directories
    BASE_DIRECTORY="/Users/curiostudio/Desktop/apd/gpt2/prob/suppress"
    OUTPUT_DIRECTORY="/Users/curiostudio/Desktop/apd/gpt2-bias/50bias_check-suppress48"



# Function to perform linear regression using log rank data within specified range
def log_rank_linear_regression(values, config=LogRankConfig):
    """
    Perform linear regression on log rank data within specified range and calculate
    the delta bias for each point before the target range.
    
    Args:
        values: List of data values to analyze
        config: Configuration parameters
        
    Returns:
        Dictionary containing regression results and delta biases
    """
    # Remove zeros and sort values in descending order
    arr = np.array(values)
    arr = arr[arr > 0]  # Remove zeros or negative values
    arr = -np.sort(-arr)  # Sort in descending order
    
    # Compute ranks and convert to log scale
    ranks = np.arange(1, len(arr) + 1)
    log_ranks = np.log(ranks)  # Using natural log
    log_vals = np.log(arr)
    
    # Extract data points with log rank within the specified range
    target_mask = (log_ranks >= config.LOG_RANK_MIN) & (log_ranks <= config.LOG_RANK_MAX)
    target_log_ranks = log_ranks[target_mask]
    target_log_vals = log_vals[target_mask]
    
    # If there are not enough data points in the target range, return early
    if len(target_log_ranks) < 2:
        return {
            "slope": None,
            "initial_bias": None,
            "delta_biases": [],
            "target_range": {
                "log_ranks": np.array([]),
                "log_vals": np.array([]),
                "fitted_vals": np.array([])
            },
            "extended_fit": {
                "log_ranks": np.array([]),
                "fitted_vals": np.array([])
            }
        }
    
    # Perform linear regression on the target range
    coeffs = np.polyfit(target_log_ranks, target_log_vals, 1)
    slope, initial_bias = coeffs
    
    # Calculate the delta bias for each point (all points, not just early ones)
    # The delta bias is the vertical distance from each point to the regression line
    regression_line = lambda x: slope * x + initial_bias
    predicted_vals = regression_line(log_ranks)
    delta_biases = log_vals - predicted_vals
    
    # Prepare delta biases with their corresponding ranks for all points
    delta_bias_data = []
    for i in range(len(log_ranks)):
        delta_bias_data.append({
            "rank": int(ranks[i]),
            "log_rank": float(log_ranks[i]),
            "log_val": float(log_vals[i]),
            "predicted_val": float(predicted_vals[i]),
            "delta_bias": float(delta_biases[i])
        })
    
    # Generate fitted lines for visualization
    target_fitted = slope * target_log_ranks + initial_bias
    
    # Create extended fit line for full visualization
    all_log_ranks = np.linspace(np.min(log_ranks), np.max(log_ranks), 100)
    extended_fit = slope * all_log_ranks + initial_bias
    
    return {
        "slope": slope,
        "initial_bias": initial_bias,
        "delta_biases": delta_bias_data,
        "target_range": {
            "log_ranks": target_log_ranks,
            "log_vals": target_log_vals,
            "fitted_vals": target_fitted
        },
        "extended_fit": {
            "log_ranks": all_log_ranks,
            "fitted_vals": extended_fit
        }
    }

# Function to visualize the results with emphasis on delta biases
def plot_log_rank_regression(log_ranks, log_vals, regression_results, title, save_path=None, config=LogRankConfig):
    """Plot the log rank regression results with focus on delta biases."""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), gridspec_kw={'height_ratios': [2, 1]})
    
    # Plot 1: Data points and regression fit
    ax1.scatter(log_ranks, log_vals, alpha=0.6, s=10, color='blue', label='Data Points')
    
    # Check if we have valid results (slope is not None)
    if regression_results["slope"] is None:
        ax1.set_title(f"{title}\nInsufficient data in {config.LOG_RANK_MIN}-{config.LOG_RANK_MAX} range", fontsize=14)
        ax1.set_xlabel("Log Rank", fontsize=12)
        ax1.set_ylabel("Log Value", fontsize=12)
        ax1.legend(loc='best', fontsize=10)
        ax1.grid(True, alpha=0.3)
        
        # Set up the second subplot (empty)
        ax2.set_xlabel("Log Rank", fontsize=12)
        ax2.set_ylabel('Delta Bias', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.text(0.5, 0.5, "No delta bias data available", 
                ha='center', va='center', transform=ax2.transAxes)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300)
        
        return fig
    
    # Highlight the target range
    target_range = regression_results["target_range"]
    if len(target_range["log_ranks"]) > 0:
        ax1.scatter(target_range["log_ranks"], target_range["log_vals"], 
                   color='red', s=40, label=f'Target Range ({config.LOG_RANK_MIN}-{config.LOG_RANK_MAX})')
    
    # Plot the extended fit line across the entire range
    if len(regression_results["extended_fit"]["log_ranks"]) > 0:
        ax1.plot(regression_results["extended_fit"]["log_ranks"], 
                regression_results["extended_fit"]["fitted_vals"], 
                linestyle='-', linewidth=2.5, color='green',
                label=f'Linear Fit (Slope={regression_results["slope"]:.3f}, Bias={regression_results["initial_bias"]:.3f})')
    
    # Shade the target region
    y_min, y_max = ax1.get_ylim()
    ax1.fill_between([config.LOG_RANK_MIN, config.LOG_RANK_MAX], y_min, y_max, color='yellow', alpha=0.1)
    
    # Add vertical lines at log rank boundaries
    ax1.axvline(x=config.LOG_RANK_MIN, color='gray', linestyle='--', alpha=0.7)
    ax1.axvline(x=config.LOG_RANK_MAX, color='gray', linestyle='--', alpha=0.7)
    
    ax1.set_title(title, fontsize=14)
    ax1.set_xlabel("Log Rank", fontsize=12)
    ax1.set_ylabel("Log Value", fontsize=12)
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Delta bias values
    if regression_results["delta_biases"]:
        # Extract delta bias values and their corresponding log ranks
        # Filter to only show points before the target range
        early_deltas = [db for db in regression_results["delta_biases"] if db["log_rank"] < config.LOG_RANK_MIN]
        
        if early_deltas:
            log_rank_positions = [db["log_rank"] for db in early_deltas]
            delta_bias_values = [db["delta_bias"] for db in early_deltas]
            
            # Plot delta bias values against log rank
            ax2.plot(log_rank_positions, delta_bias_values, 'r-o', linewidth=2, markersize=4)
            ax2.set_xlabel("Log Rank", fontsize=12)
            ax2.set_ylabel('Delta Bias', fontsize=12)
            ax2.grid(True, alpha=0.3)
            
            # Add horizontal line at delta bias = 0
            ax2.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
            
            # Color positive and negative regions
            ax2.fill_between(log_rank_positions, 0, delta_bias_values, 
                            where=(np.array(delta_bias_values) > 0), 
                            color='green', alpha=0.1)
            ax2.fill_between(log_rank_positions, delta_bias_values, 0, 
                            where=(np.array(delta_bias_values) < 0), 
                            color='red', alpha=0.1)
        else:
            ax2.text(0.5, 0.5, "No early delta bias data available", 
                    ha='center', va='center', transform=ax2.transAxes)
    else:
        ax2.set_xlabel("Log Rank", fontsize=12)
        ax2.set_ylabel('Delta Bias', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.text(0.5, 0.5, "No delta bias data available", 
                ha='center', va='center', transform=ax2.transAxes)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300)
    
    return fig

# Function to plot aggregated delta bias dynamics across all steps (FIXED VERSION)
def plot_delta_bias_dynamics(all_delta_biases, output_path, file_id, config=LogRankConfig):
    """
    Plot the sum of delta biases for each log rank across all steps.
    
    Args:
        all_delta_biases: Dictionary mapping step to delta bias data
        output_path: Path to save the plot
        file_id: Identifier for the file being processed
        config: Configuration parameters
    """
    if not all_delta_biases:
        print(f"No delta bias data available for {file_id}")
        return
    
    # First, collect all the delta biases by rank
    rank_to_delta_biases = {}
    
    for step, deltas in all_delta_biases.items():
        for delta_data in deltas:
            # Only consider points before the target range
            if delta_data["log_rank"] < config.LOG_RANK_MAX:
                rank = delta_data["rank"]
                if rank not in rank_to_delta_biases:
                    rank_to_delta_biases[rank] = []
                rank_to_delta_biases[rank].append(delta_data["delta_bias"])
    
    # Calculate the sum of delta biases for each rank
    rank_to_sum_delta = {}
    for rank, biases in rank_to_delta_biases.items():
        rank_to_sum_delta[rank] = sum(biases)
    
    # Sort by rank for plotting
    sorted_ranks = sorted(rank_to_sum_delta.keys())
    log_ranks = [np.log(r) for r in sorted_ranks]
    sum_delta_biases = [rank_to_sum_delta[r] for r in sorted_ranks]
    
    # Create the plot for sum of delta biases
    plt.figure(figsize=(12, 8))
    
    # Check if we have data to plot
    if not log_ranks or not sum_delta_biases:
        plt.text(0.5, 0.5, "No delta bias data available for plotting", 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.savefig(output_path, dpi=300)
        plt.close()
        return
    
    plt.plot(log_ranks, sum_delta_biases, 'b-o', linewidth=2, markersize=6)
    
    # Add a zero line
    plt.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    
    # Color positive and negative regions
    plt.fill_between(log_ranks, 0, sum_delta_biases, 
                    where=(np.array(sum_delta_biases) > 0), 
                    color='green', alpha=0.2)
    plt.fill_between(log_ranks, sum_delta_biases, 0, 
                    where=(np.array(sum_delta_biases) < 0), 
                    color='red', alpha=0.2)
    
    plt.title(f"{file_id} - Sum of Delta Biases Across All Steps", fontsize=14)
    plt.xlabel("Log Rank", fontsize=12)
    plt.ylabel("Sum of Delta Biases", fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add markers for key positions
    plt.axvline(x=config.LOG_RANK_MIN, color='red', linestyle='--', alpha=0.7, 
               label=f'Target Range Start ({config.LOG_RANK_MIN})')
    plt.legend()
    
    # Save the plot
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()
    
    # Also calculate and plot the average delta bias for each rank
    rank_to_avg_delta = {}
    for rank, biases in rank_to_delta_biases.items():
        rank_to_avg_delta[rank] = sum(biases) / len(biases)
    
    # Get the sorted values for average plot
    avg_delta_biases = [rank_to_avg_delta[r] for r in sorted_ranks]
    
    # Create the plot for average delta biases
    plt.figure(figsize=(12, 8))
    plt.plot(log_ranks, avg_delta_biases, 'g-o', linewidth=2, markersize=6)
    
    # Add a zero line
    plt.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    
    # Color positive and negative regions
    plt.fill_between(log_ranks, 0, avg_delta_biases, 
                    where=(np.array(avg_delta_biases) > 0), 
                    color='green', alpha=0.2)
    plt.fill_between(log_ranks, avg_delta_biases, 0, 
                    where=(np.array(avg_delta_biases) < 0), 
                    color='red', alpha=0.2)
    
    plt.title(f"{file_id} - Average Delta Bias Across All Steps", fontsize=14)
    plt.xlabel("Log Rank", fontsize=12)
    plt.ylabel("Average Delta Bias", fontsize=12)
    plt.grid(True, alpha=0.3)
    
    # Add markers for key positions
    plt.axvline(x=config.LOG_RANK_MIN, color='red', linestyle='--', alpha=0.7,
               label=f'Target Range Start ({config.LOG_RANK_MIN})')
    plt.legend()
    
    # Save the plot
    plt.tight_layout()
    avg_output_path = str(output_path).replace('.png', '_avg.png')
    plt.savefig(avg_output_path, dpi=300)
    plt.close()

# Function to process a dataset with the new approach
def process_dataset_log_rank(data_path, output_dir, config=LogRankConfig):
    """
    Process a dataset using the log rank regression approach.
    
    Args:
        data_path: Path to the CSV data file
        output_dir: Directory to save output visualizations
        config: Configuration parameters
    """
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Get the file ID for naming output files
    file_id = Path(data_path).stem
    
    # Create a folder for this specific CSV file's results
    file_output_dir = output_path / file_id
    file_output_dir.mkdir(parents=True, exist_ok=True)
    
    results = []
    all_delta_biases = {}  # Store delta biases by step
    all_data_for_csv = []  # Store all data for CSV export
    
    try:
        # Load and process data
        df = pd.read_csv(data_path)
        
        # If your data is stored as string representation of lists, convert it
        if isinstance(df["abs_delta_loss_post_ablation"].iloc[0], str):
            df["abs_delta_loss_post_ablation"] = df["abs_delta_loss_post_ablation"].apply(ast.literal_eval)
        
        # Process each row (e.g., each training step)
        for idx, row in df.iterrows():
            step = row.get("step", f"row_{idx}")
            vals = row["abs_delta_loss_post_ablation"]
            
            # Skip if vals is empty or all zeros
            if not vals or all(v == 0 for v in vals):
                continue
            
            # Remove zeros and sort values in descending order
            arr = np.array(vals)
            arr = arr[arr > 0]
            arr = -np.sort(-arr)
            
            # Compute ranks and log values
            ranks = np.arange(1, len(arr) + 1)
            log_ranks = np.log(ranks)
            log_vals = np.log(arr)
            
            # Perform log rank regression
            regression_results = log_rank_linear_regression(vals, config)
            
            # Store delta biases for this step
            if regression_results["slope"] is not None:
                # Only store delta biases for points before the target range
                early_deltas = [db for db in regression_results["delta_biases"] 
                               if db["log_rank"] < config.LOG_RANK_MIN]
                if early_deltas:
                    all_delta_biases[step] = early_deltas
                
                # Add data to CSV collection
                for delta_data in regression_results["delta_biases"]:
                    all_data_for_csv.append({
                        "file": file_id,
                        "step": step,
                        "rank": delta_data["rank"],
                        "log_rank": delta_data["log_rank"],
                        "log_val": delta_data["log_val"],
                        "predicted_val": delta_data["predicted_val"],
                        "delta_bias": delta_data["delta_bias"]
                    })
            
            # Generate output file name
            save_path = file_output_dir / f"step{step}_log_rank_regression.png"
            
            # Plot and save results
            title = f"{file_id} - Step {step} Log Rank Regression"
            fig = plot_log_rank_regression(log_ranks, log_vals, regression_results, title, save_path, config)
            
            # Save regression results if we have valid slope
            if regression_results["slope"] is not None:
                step_results = {
                    "file": file_id,
                    "step": step,
                    "slope": regression_results["slope"],
                    "initial_bias": regression_results["initial_bias"],
                    "delta_biases": regression_results["delta_biases"],
                }
                results.append(step_results)
            
            # Close figure to free memory
            plt.close(fig)
        
        # Generate and save the delta bias dynamics plot
        if all_delta_biases:
            print(f"Creating delta bias dynamics plot for {file_id} with {len(all_delta_biases)} steps")
            dynamics_path = file_output_dir / f"{file_id}_delta_bias_dynamics.png"
            plot_delta_bias_dynamics(all_delta_biases, dynamics_path, file_id, config)
        else:
            print(f"No valid delta bias data for {file_id}")
        
        # Save all data to CSV
        if all_data_for_csv:
            data_df = pd.DataFrame(all_data_for_csv)
            data_path = file_output_dir / f"{file_id}_delta_bias_data.csv"
            data_df.to_csv(data_path, index=False)
            print(f"Delta bias data saved to {data_path}")
    
    except Exception as e:
        print(f"Error processing {data_path}: {str(e)}")
        import traceback
        traceback.print_exc()
    
    return results, all_delta_biases

# Main function to run the analysis
def run_log_rank_analysis(config=LogRankConfig):
    """
    Run log rank regression analysis on all CSV files in the base directory.
    
    Args:
        config: Configuration parameters
    """
    # Create the output directory
    output_path = Path(config.OUTPUT_DIRECTORY)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Save configuration for reference
    config_summary = {
        "log_rank_min": config.LOG_RANK_MIN,
        "log_rank_max": config.LOG_RANK_MAX,
        "save_individual_results": config.SAVE_INDIVIDUAL_RESULTS,
        "save_consolidated_results": config.SAVE_CONSOLIDATED_RESULTS,
        "timestamp": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Save configuration to JSON
    import json
    with open(output_path / "analysis_config.json", 'w') as f:
        json.dump(config_summary, f, indent=2)
    
    # Find all CSV files in the base directory
    csv_files = list(Path(config.BASE_DIRECTORY).glob("**/*.csv"))
    
    all_results = []
    all_delta_biases_by_file = {}
    
    for csv_file in csv_files:
        print(f"Processing {csv_file}")
        results, file_delta_biases = process_dataset_log_rank(csv_file, config.OUTPUT_DIRECTORY, config)
        all_results.extend(results)
        all_delta_biases_by_file[csv_file.stem] = file_delta_biases
    
    # Save consolidated results if enabled and there are results
    if config.SAVE_CONSOLIDATED_RESULTS and all_results:
        # Create a consolidated CSV for all delta bias data
        all_data = []
        for file_id, step_deltas in all_delta_biases_by_file.items():
            for step, deltas in step_deltas.items():
                for delta_data in deltas:
                    all_data.append({
                        "file": file_id,
                        "step": step,
                        "rank": delta_data["rank"],
                        "log_rank": delta_data["log_rank"],
                        "log_val": delta_data["log_val"],
                        "predicted_val": delta_data["predicted_val"],
                        "delta_bias": delta_data["delta_bias"]
                    })
        
        # Save consolidated delta bias data
        if all_data:
            consolidated_df = pd.DataFrame(all_data)
            results_path = output_path / "consolidated_delta_bias_data.csv"
            consolidated_df.to_csv(results_path, index=False)
            print(f"Consolidated delta bias data saved to {results_path}")
    
    print(f"Processed {len(csv_files)} datasets")
    return all_results, all_delta_biases_by_file

# Example of how to run with customized configuration
def main():
    # Default configuration
    config = LogRankConfig()
    
    # Uncomment and modify these lines to customize the configuration
    # config.LOG_RANK_MIN = 3.5
    # config.LOG_RANK_MAX = 4.0
    
    # Run the analysis
    results, all_delta_biases = run_log_rank_analysis(config)
    
    return results, all_delta_biases

# Run the analysis
results, all_delta_biases = main()

In [ ]:
#或许会用到的loss密度分布图
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import ast

def run_kde_analysis(root_dir: str, output_dir: str=None):
    plt.rcParams.update({
        'font.size': 20,
        'axes.titlesize': 28,
        'axes.labelsize': 25,
        'xtick.labelsize': 20,
        'ytick.labelsize': 20,
        'legend.fontsize': 20
    })
    
    sns.set_palette("tab10")
    
    #selected_steps = ["0", "14000", "34000", "54000", "64000", "84000", "104000", "114000", "124000"]
    selected_steps = ["last"]
    
    root_path = Path(root_dir)
    if output_dir:
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
    
    results = {}
    
    csv_files = list(root_path.rglob("500_all.csv"))
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        df["abs_delta_loss_post_ablation"] = df["abs_delta_loss_post_ablation"].apply(ast.literal_eval)
        df["step"] = df["step"].astype(str)
        
        df = df[df['step'].isin(selected_steps)]
        
        rel_id = str(csv_file.relative_to(root_path).parent).replace("/", "_")
        
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        for _, row in df.iterrows():
            vals = row["abs_delta_loss_post_ablation"]
            arr = np.array(vals)
            arr = arr[arr > 0]
            if len(arr) < 5:
                continue
            sns.kdeplot(arr, label=f"{row['step']}", linewidth=3.5)
        
        plt.xlabel("ΔLoss")
        plt.ylabel("Density")
        
        #这里调节x轴段范围
        #plt.xlim(0, 0.0007)
        
        plt.legend(loc='upper right')
        plt.tight_layout()
        
        if output_dir:
            save_kde = output_path / f"{rel_id}_selected_steps_kde.png"
            plt.savefig(save_kde,dpi=200)
        
        results[rel_id] = plt.gcf()
        plt.close()
    
    return results

# root_dir = "your_data_directory"
# output_dir = "output_directory"
# kde_plots = run_kde_analysis(root_dir)
# list(kde_plots.keys())
# kde_plots['your_key']

In [ ]:
#以此往下是一阶导折点图，改最下面的参数（这是原始版本，我给文章发图的版本好像没保存或是按错删了不见了…不过只用改下尺寸就行没关系，后面我加的都是二阶导的）
# check derivatives by steps
# get rank and values

def rank_val(values)->tuple[np.array,np.array]:
    """Analyze the distribution and identify both transition points using a unified approach. """
    arr = np.array(values)
    arr = arr[arr > 0]
    sorted_vals = np.sort(arr)[::-1]
    ranks = np.arange(1, len(sorted_vals)+1)
    return np.log(ranks), np.log(sorted_vals)


def plot_slope(
    log_ranks: np.ndarray,
    log_vals: np.ndarray,
    window_size: int = 50,
) -> dict:
    """Get first derivatives of each step."""
    n = len(log_ranks)
    
    # Apply rank percentile constraints
    max_idx = min(int(len(log_ranks)), n - window_size)
    # Dynamic window size
    w = min(window_size, max(5, (max_idx - window_size) // 5))
    # Calculate first derivatives (slopes)
    derivatives, idxs = [], []
    for i in range(window_size, max_idx - w + 1):
        slope, _, _, _, _ = stats.linregress(
            log_ranks[i : i + w], log_vals[i : i + w]
        )
        derivatives.append(slope)
        idxs.append(i)
    derivatives = np.array(derivatives)
    return derivatives


def run_analysis(
    root_dir: str, 
    output_dir: str, 
    ylim: tuple = None,
    window_size: int = 50,
):
    """Batch process all 500_all.csv files in the directory,"""
    root_path = Path(root_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    
    # Find all CSV files
    csv_files = list(root_path.rglob("500_all.csv"))
    
    
    for csv_file in csv_files:
        # Read and preprocess the CSV
        df = pd.read_csv(csv_file)
        df["abs_delta_loss_post_ablation"] = df["abs_delta_loss_post_ablation"].apply(ast.literal_eval)
        df["step"] = df["step"].astype(str)
        rel_id = str(csv_file.relative_to(root_path).parent).replace("/", "_")
        
        
        # Analyze each training step
        for _, row in df.iterrows():
            vals = row["abs_delta_loss_post_ablation"]
            log_ranks, log_vals = rank_val(vals)
            derivatives = plot_slope(log_ranks, log_vals)
            plt.plot(log_ranks[:-window_size],derivatives)
            plt.show()
            
            # Apply the unified approach with custom parameters
            
            
            

In [ ]:
def plot_slope(
    log_ranks: np.ndarray,
    log_vals: np.ndarray,
    window_size: int = 2,
) -> tuple[np.ndarray, np.ndarray]:
    """Get first derivatives of each step.
    
    Returns:
        tuple: (derivatives, indices) where derivatives are the slopes and 
               indices are the corresponding positions in the original array
    """
    n = len(log_ranks)
    # Apply rank percentile constraints
    max_idx = min(int(len(log_ranks)), n - window_size)
    # Dynamic window size
    w = min(window_size, max(5, (max_idx - window_size) // 5))

    # Calculate first derivatives (slopes)
    derivatives, idxs = [], []
    for i in range(window_size, max_idx - w + 1):
        slope, _, _, _, _ = stats.linregress(
            log_ranks[i : i + w*i], log_vals[i : i + w*i]
        )
        derivatives.append(slope)
        idxs.append(i)
    return np.array(derivatives), np.array(idxs)

def run_analysis(
    root_dir: str,
    output_dir: str,
    ylim: tuple = None,
    window_size = 5,
    step_lst = [],
):
    """Batch process all 500_all.csv files in the directory."""
    root_path = Path(root_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    # Find all CSV files
    csv_files = list(root_path.rglob("500_all.csv"))
    for csv_file in csv_files:
        # Read and preprocess the CSV
        df = pd.read_csv(csv_file)
        df["abs_delta_loss_post_ablation"] = df["abs_delta_loss_post_ablation"].apply(ast.literal_eval)
        df["step"] = df["step"].astype(str)
        rel_id = str(csv_file.relative_to(root_path).parent).replace("/", "_")
        
        # Analyze each training step
        
        for _, row in df.iterrows():
            vals = row["abs_delta_loss_post_ablation"]
            step = str(row["step"])
            log_ranks, log_vals = rank_val(vals)
            if step in step_lst:
                derivatives, idxs = plot_slope(log_ranks, log_vals, window_size)
                # Use the correct indices for plotting
                plt.plot(log_ranks[idxs], derivatives,label = step)
            
        plt.title(f"{rel_id}")
        plt.xlabel("Log rank")
        plt.ylabel("Slope")
        plt.ylim(ylim)
        plt.legend()
        plt.show()
        # Apply the unified approach with custom parameterswindow_size: int = 10,

In [ ]:
file_metrics = run_analysis(
    root_dir=f"{ROOT_dir}/results/selection/neuron/longtail_50",
    output_dir=f"{ROOT_dir}/fig/loss_slope",
    ylim=[-1.5,0],
    window_size = 5,
    step_lst = ["4000","24000", "64000","143000"]
)
